# News Mood Analysis 

Observations:

1. On aggregate, the New York Times had the most negative tweets and CBS the most positive over the period analyzed. 
2. All of the outlets had a high level of variation in the compound scores. 
3. There was no clear trend in the aggregate polarity of tweets over time in the period analyzed. Plotting this data over an extended period of time could be very interesting.

In [1]:
# Dependencies
import tweepy
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
#from datetime import datetime (not needed right now)

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [2]:
# Twitter API Keys
consumer_key = "Ed4RNulN1lp7AbOooHa9STCoU"
consumer_secret = "P7cUJlmJZq0VaCY0Jg7COliwQqzK0qYEyUF9Y0idx4ujb3ZlW5"
access_token = "839621358724198402-dzdOsx2WWHrSuBwyNUiqSEnTivHozAZ"
access_token_secret = "dCZ80uNRbFDjxdU2EckmNiSckdoATach6Q8zb7YYYE5ER"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
#loop through search terms
target_users = ["BBC", "CBS", "CNN", "Fox", "NYT"]

dictionaries = [{},{},{},{},{}]

for x in range(5):
    
    # Lists to hold sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    source_list = []
    tweets_text = []
    tweet_dates = []
    tweets_ago = []
    tweet_counter = 0

    # Grab 100 tweets
    public_tweets = api.user_timeline(target_users[x], count=100, result_type="recent", lang = "en")

   # Loop through all tweets
    for tweet in public_tweets:

        # Run Vader Analysis on each tweet
        compound = analyzer.polarity_scores(tweet["text"])["compound"]
        pos = analyzer.polarity_scores(tweet["text"])["pos"]
        neu = analyzer.polarity_scores(tweet["text"])["neu"]
        neg = analyzer.polarity_scores(tweet["text"])["neg"]
        tweet_text = tweet["text"]
        tweet_date = tweet["created_at"]
        
        # Add each value to the appropriate array
        compound_list.append(compound)
        positive_list.append(pos)
        negative_list.append(neg)
        neutral_list.append(neu)
        source_list.append(user)
        tweets_text.append(tweet_text)
        tweet_dates.append(tweet_date)
        tweets_ago.append(tweet_counter)
        tweet_counter += 1

    # Store the Average Sentiments
    dictionaries[x] = {"Compound": compound_list,
                 "Positive": positive_list,
                 "Neutral": negative_list,
                 "Negative": neutral_list,
                 "Source":source_list,
                 "Tweet Text": tweets_text,
                 "Tweet Date": tweet_dates,
                 "Tweets Ago": tweets_ago}


NameError: name 'user' is not defined

In [ ]:
#pull analysis of each source into a separate data frame
bbc_dict = dictionaries[0]
bbc_df = pd.DataFrame(bbc_dict)

cbs_dict = dictionaries[1]
cbs_df = pd.DataFrame(cbs_dict)

cnn_dict = dictionaries[2]
cnn_df = pd.DataFrame(cnn_dict)

fox_dict = dictionaries[3]
fox_df = pd.DataFrame(fox_dict)

nyt_dict = dictionaries[4]
nyt_df = pd.DataFrame(nyt_dict)

compiled_df = [bbc_df, cbs_df, cnn_df, fox_df, nyt_df]

#save a csv of all data retrieved
data_to_store = pd.DataFrame(compiled_df)
data_to_store.to_csv("news_mood_data.csv")



In [ ]:
#create the scatterplot
colors = ["coral", "firebrick", "green", "gold", "dodgerblue"]
color_counter = 0
labels = ["BBC", "CBS", "CNN", "Fox", "NYT"]
label_counter = 0

for df in compiled_df:   
        
    # Build the scatter plots for each source
    plt.scatter(df["Tweets Ago"], 
                df["Compound"], 
                s = 25,
                c=colors[color_counter], 
                edgecolor="black", linewidths=1, marker="o", 
                alpha=0.8, label= labels[label_counter])

    color_counter += 1
    label_counter += 1


# Incorporate the other graph properties
plt.title("Sentiment Analysis of Media Tweets (12/15/17)")
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")
plt.xlim((0,100))
plt.ylim(-1.1, 1.1)
plt.grid(True)
sns.set(style = "darkgrid")

# Create a legend
lgnd = plt.legend(mode="Expanded", 
                  numpoints=1, scatterpoints=1, 
                  loc="upper right", title="Media Sources",
                  bbox_to_anchor=(1.3, 1),
                  labelspacing=0.5)
lgnd.legendHandles[0]._sizes = [30]
lgnd.legendHandles[1]._sizes = [30]
lgnd.legendHandles[2]._sizes = [30]
lgnd.legendHandles[3]._sizes = [30]
lgnd.legendHandles[4]._sizes = [30]

plt.savefig("sentiment_scatter.png")
plt.show()

In [ ]:
bbc_df["Compound"].mean()

In [4]:
#create bar chart data
agg_scores = []

for x in range(5):
    score = compiled_df[x]["Compound"].mean()
    agg_scores.append(score)

#clear the plt
plt.gcf().clear()

#create the bar chart
x_axis = np.arange(len(target_users))

# Tell matplotlib that we will be making a bar chart
# Users is our y axis and x_axis is, of course, our x axis
# We apply align="edge" to ensure our bars line up with our tick marks
plt.bar(x_axis, agg_scores, color= colors, alpha=0.5, align="edge")

tick_locations = [value+0.4 for value in x_axis]
plt.xticks(tick_locations, target_users)

plt.title("Overall Media Sentiment based on Twitter")
plt.xlabel("Media Outlet")
plt.ylabel("Tweet Polarity")

# Sets the y limits of the current chart
plt.ylim(-0.2, 0.5)

sns.set()
plt.savefig("sentiment_bar.png")
plt.show()


NameError: name 'compiled_df' is not defined